# EJERCICIO 7
A partir de análisis clínicos y de la edad y el sexo de pacientes de una clínica ubicada en el noreste de Andhra Pradesh, India, se intentará obtener un clasificador automático que sirva para diagnosticar a pacientes con problemas de hígado.
Para esto, se recabaron muestras de ocho análisis distintos realizados a 579 pacientes que, junto con su edad y sexo, se dividieron en dos grupos: 414 de ellos diagnosticados con problemas de hígado por expertos en el área mientras que los 165 restantes fueron señalados como exentos de ese problema.
Los 11 atributos que constituyen una muestra son los
indicados en la tabla de la derecha. Todos son atributos son
valores numéricos continuos a excepción del atributo “Sexo”,
en donde el valor 1 representa “HOMBRE” y el valor 2
representa “MUJER”, y del atributo “Diagnóstico”, donde el valor 1 representa “CON PROBLEMA DE HÍGADO” mientras que el valor 2 representa “SIN PROBLEMA DE HÍGADO”.
Utilice perceptrones o una red neuronal artificial (según resulte más conveniente). Informe el motivo por el que se eligió el tipo de clasificador. Detalle la arquitectura y los parámetros usados en su entrenamiento (según corresponda). Documente todos los intentos realizados.
Para el entrenamiento emplee sólo el 90% de las muestras disponibles de cada tipo. Informe la matriz de confusión que produce el mejor clasificador obtenido al evaluarlo con las muestras de entrenamiento e indique la matriz que ese clasificador produce al usarlo sobre el resto de las muestras reservadas para prueba.

$$
\begin{array}{|c|c|}
\hline 1 & Edad \\
\hline 2 & Sexo \\
\hline 3 & Bilirrubina Total \\
\hline 4 & Bilirrubina Directa \\
\hline 5 & Fosfatasa Alcalina \\
\hline 6 & Alanina Aminotransferasa \\
\hline 7 & Aspartato Aminotransferasa \\
\hline 8 & Proteínas Total \\
\hline 9 & Albúmina \\
\hline 10 & Relación Albúmina/Globulina \\
\hline 11 & Diagnóstico (valor a predecir) \\
\hline 
\end{array}
$$

In [1]:
import pyximport; pyximport.install()
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import mpld3
%matplotlib inline
mpld3.enable_notebook()
from utils.cperceptron import Perceptron
from utils.cbackpropagation import ANN #, Identidad, Sigmoide
import utils.patrones as magia

def progreso(ann, X, T, y=None, n=-1, E=None):
    if n % 20 == 0:
        print("Pasos: {0} - Error: {1:.32f}".format(n, E)) 
def progresoPerceptron(perceptron, X, T, n):
    y = perceptron.evaluar(X)
    incorrectas = (T != y).sum()
    print("Pasos: {0}\tIncorrectas: {1}\n".format(n, incorrectas))

In [2]:
#Cargo datos
higado = np.load('resources/higado.npy')

#muestras = higado[:, :-1]
muestras = np.hstack((higado[:,0].reshape(-1,1), higado[:,2:-1]))
# Atributo 'sexo':
#     muestras[:, 1] == 1 --> HOMBRE
#     muestras[:, 1] == 2 --> MUJER

diagnostico = (higado[:, -1] != 2).astype(np.int8)
# diagnostico == 1 --> CON PROBLEMA DE HÍGADO
# diagnostico == 0 --> SIN PROBLEMA DE HÍGADO

In [3]:
#Armo Patrones
clases, patronesEnt, patronesTest = magia.generar_patrones(magia.escalar(muestras),diagnostico,90)
X, T = magia.armar_patrones_y_salida_esperada(clases,patronesEnt)
Xtest, Ttest = magia.armar_patrones_y_salida_esperada(clases,patronesTest)

utils/patrones.py:84: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  X = np.vstack(patrones[c] for c in clases)


In [8]:
# Esto es para poder usar Cython y que sea mas rapido
X = X.astype(np.float)
Xtest = Xtest.astype(np.float)
TT = T[:,0].copy(order='C')
TT = TT.astype(np.int8)

## Intento con Perceptrones, pero no funciona

In [9]:
#Entrenamiento
p1 = Perceptron(X.shape[1])
p1.reiniciar()
I1 = p1.entrenar(X, TT, max_pasos=100000, callback=progresoPerceptron, frecuencia_callback=50000)
print("Pasos:{0}".format(I1))

Pasos: 50000	Incorrectas: 148

Pasos: 100000	Incorrectas: 148

Pasos:100000


In [10]:
#Evaluo
print("Errores:{0} de {1}\n".format((p1.evaluar(Xtest) != Ttest[:,0]).sum(), Ttest.shape[0]))

Errores:17 de 59



## Ahora intento con BackPropagation

In [11]:
# Crea la red neuronal
ocultas = 20 #2,3,5,10,20
entradas = X.shape[1]
salidas = T.shape[1]
ann = ANN(entradas, ocultas, salidas)
ann.reiniciar()

In [12]:
#Entreno
E, n = ann.entrenar_rprop(X, T, min_error=0, max_pasos=100000, callback=progreso, frecuencia_callback=50000)
print("\nRed entrenada en {0} pasos con un error de {1:.32f}".format(n, E))

Pasos: 50000 - Error: 0.06005377856659139396144908573660
Pasos: 100000 - Error: 0.05094376337264409221328165244813

Red entrenada en 100000 pasos con un error de 0.05094376337264409221328165244813


In [13]:
Y = (ann.evaluar(Xtest) >= 0.5).astype(np.float32)

In [14]:
magia.matriz_de_confusion(Ttest,Y)

(array([[ 6., 11.,  0.],
        [14., 28.,  0.]]), [array([1, 0]), array([0, 1])])